In [1]:
# 温度、湿度、温湿度都绘制
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Times New Roman']  # 指定默认
plt.rcParams['axes.unicode_minus'] = False
# ===== 配置 =====

In [ ]:
# 每年的平均值nc
import numpy as np
import xarray as xr
import os
stl = 'stl4'
swvl = 'swvl4'
# 设置温度和湿度数据的文件夹路径
temp_folder = f"D:/Permafrost/temp/stl/{stl}"  # 温度数据文件夹路径
humidity_folder = f"D:/Permafrost/temp/swvl/{swvl}"  # 湿度数据文件夹路径
temp_l = os.listdir(temp_folder)
humidity_l = os.listdir(humidity_folder)
# 初始化一个空的列表来存储每年数据的均值
temp_annual_means = []
humidity_annual_means = []

# 遍历每年的数据文件
for year in range(25):
    # 温度文件路径
    temp_file = os.path.join(temp_folder, temp_l[year])
    # 湿度文件路径
    humidity_file = os.path.join(humidity_folder, humidity_l[year])
    # 读取.nc文件
    temp_ds = xr.open_dataset(temp_file)
    humidity_ds = xr.open_dataset(humidity_file)

    # 假设温度和湿度变量名为 'temperature' 和 'humidity'
    temperature = temp_ds[stl]  # 读取温度数据 (time, lat, lon)
    humidity = humidity_ds[swvl]  # 读取湿度数据 (time, lat, lon)

    # 计算每年（12个月）的均值：这里假设时间维度是按月排列的
    temp_annual_mean = temperature.mean(dim=['valid_time'])  # 对时间维度求均值 (lat, lon)
    humidity_annual_mean = humidity.mean(dim=['valid_time'])  # 对时间维度求均值 (lat, lon)

    # 将计算得到的年度均值存储到列表中
    temp_annual_means.append(temp_annual_mean)
    humidity_annual_means.append(humidity_annual_mean)

    # 关闭文件
    temp_ds.close()
    humidity_ds.close()
print('25年数据均值计算完成')
# 创建新的Dataset存储25年均值数据'longitude', 'latitude'
new_ds = xr.Dataset(
    {
        'temperature_25year_mean': (['time', 'latitude', 'longitude'], temp_annual_means),
        'humidity_25year_mean': (['time', 'latitude', 'longitude'], humidity_annual_means)
    },
    coords={
        'time': np.array(range(1997, 2022)),
        'latitude': temp_ds['latitude'],
        'longitude': temp_ds['longitude']
    }
)

# 保存为新的.nc文件
new_ds.to_netcdf('D:/Permafrost/temp/y_m_stl_swvl/layer4_stl_swvl_25year_ann_mean.nc')
# 关闭Dataset
new_ds.close()
print("25年均值文件保存完成")

In [ ]:
# 整合300值nc
import numpy as np
import xarray as xr
import os
stl = 'stl1'
swvl = 'swvl1'
# 设置温度和湿度数据的文件夹路径
temp_folder = f"D:/Permafrost/temp/stl/{stl}"  # 温度数据文件夹路径
humidity_folder = f"D:/Permafrost/temp/swvl/{swvl}"  # 湿度数据文件夹路径
temp_fl = [f"D:/Permafrost/temp/stl/{stl}/"+i for i in os.listdir(temp_folder)]
humidity_fl = [f"D:/Permafrost/temp/swvl/{swvl}/"+i for i in os.listdir(humidity_folder)]
###
print(temp_fl)
ds_t = [xr.open_dataset(i) for i in temp_fl]
ds_h = [xr.open_dataset(i) for i in humidity_fl]
ds_t_a = xr.concat(ds_t, dim='valid_time')
ds_h_a = xr.concat(ds_h, dim='valid_time')
ds_t_a.to_netcdf('D:/Permafrost/temp/stl/stl1_300.nc')
ds_h_a.to_netcdf('D:/Permafrost/temp/swvl/swvl1_300.nc')
ds_t_a.close()
ds_h_a.close()

In [12]:
# 空间STMI计算，高效版
import numpy as np
import xarray as xr
t_ = 'stl4'
h_ = 'swvl4'
# ★ 如果数据很大，建议开启按 time 维分块（可按实际数据调整块大小）
t_ds = xr.open_dataset(f"D:/Permafrost/temp/stl/{t_}_300.nc",  chunks={"time": 128}) - 273.15
h_ds = xr.open_dataset(f"D:/Permafrost/temp/swvl/{h_}_300.nc", chunks={"time": 128})
# 取变量
T = t_ds[t_]    # (time, y, x) 或 (time, lat, lon)
H = h_ds[h_]   # (time, y, x) 或 (time, lat, lon)
# 对齐（保证 time/空间网格完全一致；若两者坐标完全相同会是无操作）
T, H = xr.align(T, H, join="exact")
# 1) 计算每个像元在“时间维”的湿度最大值
H_max = H.max(dim="valid_time", skipna=True)                # (lat, lon)
# 2) 仅在 T>0 时替换湿度：H_new = 2*H_max - H；否则保持原值
H_new = xr.where(T > 0, 2 * H_max - H, H)             # (time, lat, lon)
# 3) 温度与新湿度乘积
prod = T * H                                   # (time, lat, lon)
# 4) 按像元在时间维标准化到 [-1, 1]
# pmin = prod.min(dim="valid_time", skipna=True)
# pmax = prod.max(dim="valid_time", skipna=True)
# den  = pmax - pmin
# norm = xr.where(den != 0, (2 * (prod - pmin) / den) - 1, np.nan)  # 避免除零
norm = prod.transpose("valid_time", "latitude", "longitude")
# 5) 组织与保存（沿用原本坐标与维度，避免手写 dims/coords 引发名称不一致）
out = norm.to_dataset(name="normalized_temp_humidity_product")
out.attrs["crs"] = "EPSG:4326"
# ★ 压缩与填充值设置（更小的文件、更快的写入；按需调整 complevel）
comp = dict(zlib=True, complevel=4, _FillValue=None)
encoding = {"normalized_temp_humidity_product": comp}
out.to_netcdf("D:/Permafrost/temp/y_m_stl_swvl/stmi_layer4.nc", encoding=encoding)
# 关闭文件（可选，to_netcdf 后会触发计算）
t_ds.close(); h_ds.close(); out.close()